In [12]:
import importlib.util

if importlib.util.find_spec("petsc4py") is not None:
    import dolfinx

    if not dolfinx.has_petsc:
        print("This demo requires DOLFINx to be compiled with PETSc enabled.")
        exit(0)
    from petsc4py.PETSc import ScalarType  # type: ignore
else:
    print("This demo requires petsc4py.")
    exit(0)

from mpi4py import MPI

# +
import numpy as np

import ufl
from dolfinx import fem, io, mesh, plot
from dolfinx.mesh import create_box
from dolfinx.fem.petsc import LinearProblem
from dolfinx.fem import FunctionSpace

from ufl import ds, dx, grad, inner

Creating the mesh and function space

In [19]:
# msh1 = mesh.create_unit_cube(
#     comm=MPI.COMM_WORLD,
#     nx=32,
#     ny=16,
#     nz=8,
#     cell_type=mesh.CellType.tetrahedron  # Or mesh.CellType.hexahedron
# )

msh1 = create_box(
    comm=MPI.COMM_WORLD,
    points=[(0.0, 0.0, 0.0), (3.0, 2.0, 1.0)],
    n=[30, 20, 10],
    cell_type=mesh.CellType.tetrahedron,
)
print(msh1)

It is recommended in papers to use Nedric elements (I don't know much about these)

In [ ]:
print(type(msh1))
V = fem.FunctionSpace(msh1, element= 'N1curl', cppV=1) #"N1curl", 1)
# Define basis and bilinear form
u = fem.TrialFunction(V)
v = fem.TestFunction(V)

<class 'dolfinx.mesh.Mesh'>


AttributeError: 'int' object has no attribute 'mesh'

 To apply the Dirichlet boundary conditions, we find the mesh facets
 (entities of topological co-dimension 1) that lie on the boundary
 $\Gamma_D$ using {py:func}`locate_entities_boundary
<dolfinx.mesh.locate_entities_boundary>`. The function is provided
 with a 'marker' function that returns `True` for points `x` on the
boundary and `False` otherwise.

In [ ]:
facets = mesh.locate_entities_boundary(
    msh,
    dim=(msh.topology.dim - 1),
    marker=lambda x: np.isclose(x[0], 0.0) | np.isclose(x[0], 2.0),
)

We now find the degrees-of-freedom that are associated with the
boundary facets using {py:func}`locate_dofs_topological
<dolfinx.fem.locate_dofs_topological>`:

and use {py:func}`dirichletbc <dolfinx.fem.dirichletbc>` to create a
{py:class}`DirichletBC <dolfinx.fem.DirichletBC>` class that
represents the boundary condition:

In [ ]:
dofs = fem.locate_dofs_topological(V=V, entity_dim=1, entities=facets)
bc = fem.dirichletbc(value=ScalarType(0), dofs=dofs, V=V)


Next we define the variationaly formulation:

$$ \nabla \times  \left(\frac{1}{\mu} \nabla \times \mathbf{E}(\mathbf{r},\omega) \right) = \omega^2 \varepsilon \mathbf{E}(\mathbf{r},\omega) $$
$$ \nabla \times E \times n = g_N \quad \partial \Omega_N$$
$$ E \times n = g_D \quad \partial \Omega _D $$

Through integration by parts we get,
$$ \int_\Omega (\nabla \times \underline{E}) \cdot \nabla \times \underline{v}\ d\Omega - \int_{\partial\Omega_N}   (\nabla \times \underline{E}) \cdot (\nabla \times \underline{v} \ \partial\Omega_N - \omega^2 \varepsilon \int_\Omega \underline{E}\underline{v}\ d\Omega= 0$$ 
,

by using: https://en.wikipedia.org/wiki/Triple_product